In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from iuvs import io

In [4]:
muvdarks = list(io.get_l1a_darks('muv'))

Searching for *muvdark*.fits.gz


In [8]:
from IPython.parallel import Client
c = Client()
lb = c.load_balanced_view()

In [18]:
def search_ff_dark(fname):
    from iuvs import io
    data = io.IUVS1AReader(str(fname))
    if data.img_header['NAXIS1'] == 1024:
        if data.img_header['NAXIS2'] == 1024:
            return fname

In [23]:
res = lb.map_async(search_ff_dark, muvdarks)

In [27]:
import time
while not res.ready():
    time.sleep(20)
    print(100*res.progress/len(muvdarks),'%')

66.80377449794338 %
76.74812484877813 %
85.6278732155819 %
95.33026857004597 %
100.0 %


In [30]:
import pandas as pd
s = pd.Series(res.result).dropna()

In [63]:
import datetime as dt
import os
class IUVS_Filename:
    def __init__(self, fname):
        tokens = os.path.basename(fname).split('_')
        self.mission, self.instrument = tokens[:2]
        self.level = tokens[2]
        self.phase = tokens[3]
        self.timestr, self.version = tokens[4:6]
        self.revision = tokens[6].split('.')[0]
        phasetokens = self.phase.split('-')
        self.tokens = tokens
        self.phase, self.cycle, self.mode, self.channel = phasetokens
        self.time = dt.datetime.strptime(self.timestr,
                                         '%Y%m%dT%H%M%S')

In [64]:
f = IUVS_Filename(str(s.tolist()[0]))

In [65]:
f.tokens

['mvn',
 'iuv',
 'l1a',
 'IPH2-cycle00103-mode040-muvdark',
 '20140527T154453',
 'v00',
 'r00.fits.gz']

In [74]:
s = s.map(lambda x: os.path.basename(str(x)))

In [75]:
s.sort()

In [77]:
df = pd.DataFrame(s)

In [80]:
df.columns = ['basename']

In [83]:
df['cycle'] = df.basename.map(lambda x: IUVS_Filename(x).cycle[:5])

In [84]:
df.cycle.value_counts()

cycle    513
orbit      2
dtype: int64

In [24]:
data = io.IUVS1AReader(str(ff_muvdarks[0]))

In [25]:
header = data.img_header

In [27]:
data.fname

'mvn_iuv_l1a_IPH2-cycle00103-mode040-muvdark_20140527T154453_v00_r00.fits.gz'

In [26]:
header

SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    2 /                                                
NAXIS1  =                 1024 /                                                
NAXIS2  =                 1024 /                                                
EXTEND  =                    T /Extensions may be present                       
COMMENT MAVEN IUVS Level 1A Data Product                                        
COMMENT IUVS Principal Investigator N. Schneider                                
COMMENT Laboratory for Atmospheric and Space Physics                            
COMMENT University of Colorado Boulder                                          
COMMENT 1234 Innovation Drive, Boulder CO 80303                                 
COMMENT MAVEN Mission scientific and model results are open to all.             
COMMENT Users should contact

In [4]:
%matplotlib qt

In [21]:
from matplotlib import rcParams
rcParams['image.interpolation'] = 'None'

In [22]:
data.plot_img_data()

In [29]:
s = pd.Series(ff_muvdarks)

In [32]:
s.to_pickle('ff_muvdarks.pickle')

In [5]:
s = pd.io.pickle.read_pickle('ff_muvdarks.pickle')

In [6]:
data = io.IUVS1AReader(str(s[0]))

In [7]:
data.plot_img_data()

In [7]:
data.img_header

 [astropy.io.fits.verify]



SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    2 /                                                
NAXIS1  =                 1024 /                                                
NAXIS2  =                 1024 /                                                
EXTEND  =                    T /Extensions may be present                       
COMMENT MAVEN IUVS Level 1A Data Product                                        
COMMENT IUVS Principal Investigator N. Schneider                                
COMMENT Laboratory for Atmospheric and Space Physics                            
COMMENT University of Colorado Boulder                                          
COMMENT 1234 Innovation Drive, Boulder CO 80303                                 
COMMENT MAVEN Mission scientific and model results are open to all.             
COMMENT Users should contact

In [8]:
from skimage.exposure import equalize_hist

In [9]:
equal = equalize_hist(data.img)

In [11]:
plt.imshow(equal)

In [15]:
from IPython.parallel import Client

In [16]:
c = Client()

In [54]:
dview = c[:]
lb = c.load_balanced_view()

In [59]:
%%px
from iuvs import io
import numpy as np

In [90]:
int_times = []
errors = []
def get_int_times(fpath):
    try:
        int_time = io.IUVS1AReader(str(fpath)).img_header['INT_TIME']
    except ValueError:
        return
    return fpath,int_time

In [91]:
result = lb.map_async(get_int_times, s.tolist())

In [92]:
result

<AsyncMapResult: get_int_times>

In [93]:
results = result.get()

In [95]:
cleaned = []
for item in results:
    if item:
        cleaned.append(item)

In [100]:
import os
index = [str(i) for i,j in cleaned]
times = [j for i,j in cleaned]

In [102]:
s = pd.Series(times, index=index)

In [103]:
s.value_counts()

60.0    226
10.2    219
6.0     214
14.4     15
dtype: int64

In [109]:
t6 = s[s < 10].index[0]

In [112]:
t10 = s[(s>10) & (s<14)].index[0]

In [113]:
t14 = s[(s>14) & (s<20)].index[0]

In [115]:
t60 = s[s>59].index[0]

In [116]:
t60

'/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_IPH2-cycle00103-mode040-muvdark_20140527T154453_v00_r00.fits.gz'

In [ ]:
def plot_img_data(img,ax):
    ax.imshow(img)
    ax.set_title("{}, {}, {}".format())

In [120]:
equalled = []
fig, axes = plt.subplots(2,2)
axes = axes.flatten()
for ax,fname in zip(axes, [t6,t10,t14,t60]):
    print(fname)
    data = io.IUVS1AReader(fname)
    ax.imshow(data.img)
    ax.set_title("{}, {}, {}".format(data.fname.channel,
                                       data.fname.phase,
                                       data.img_header['INT_TIME']))

/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_IPH2-cycle00022-mode050-muvdark_20140426T023038_v00_r00.fits.gz
/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_early-orbit00003-mode003-muvdark_20140925T202812_v00_r00.fits.gz


TypeError: Invalid dimensions for image data

In [121]:
data = io.IUVS1AReader(t10)

In [122]:
data.img_header

 [astropy.io.fits.verify]



SIMPLE  =                    T /Primary Header created by MWRFITS v1.11         
BITPIX  =                   32 /                                                
NAXIS   =                    3 /                                                
NAXIS1  =                 1024 /                                                
NAXIS2  =                   36 /                                                
NAXIS3  =                    2 /                                                
EXTEND  =                    T /Extensions may be present                       
COMMENT MAVEN IUVS Level 1A Data Product                                        
COMMENT IUVS Principal Investigator N. Schneider                                
COMMENT Laboratory for Atmospheric and Space Physics                            
COMMENT University of Colorado Boulder                                          
COMMENT 1234 Innovation Drive, Boulder CO 80303                                 
COMMENT MAVEN Mission scient

In [123]:
data.img.shape

(2, 36, 1024)

In [124]:
plt.figure()
plt.imshow(data.img[0])

In [ ]:
pd.read_tab

In [19]:
l1a_metadata = pd.read_table("/maven_iuvs/stage/products/level1a/IUVS_L1A_index_summary.txt",
              skiprows=26, skipinitialspace=True, sep=' ',index_col=0)

In [57]:
ff = l1a_metadata[(l1a_metadata.NX == 1024) & (l1a_metadata.NY == 1024)]

In [56]:
l1a_metadata['is_dark'] = l1a_metadata.filename.map(lambda x: 'dark' in x)

In [89]:
ff[ff.is_dark][['INT_TIME','MCP_HV']]

,INT_TIME,MCP_HV
filenum,,
6610,60000,0
6625,14400,0
6626,14400,0
6627,14400,0
6628,14400,0


In [62]:
ff.index = ff.index.astype('int')

In [76]:
from matplotlib import rcParams
rcParams['image.interpolation'] = None

In [80]:
roi_indexes = [6610, 6627]

In [83]:
plt.colorbar?

In [99]:
fig, axes = plt.subplots(ncols=2, nrows=2)
for ax, ind in zip(axes.flatten()[:2], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img #/ data.img_header['INT_TIME']
    im = ax.imshow(img)
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
for ax, ind in zip(axes.flatten()[2:], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img / data.img_header['INT_TIME']
    im = ax.imshow(img)
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
fig.suptitle('Original darks, then scaled by integration time. (to DN/s)')
plt.savefig('Original_darks.png', dpi=150)

In [91]:
from skimage.exposure import rescale_intensity

In [100]:
fig, axes = plt.subplots(ncols=2, nrows=2)
for ax, ind in zip(axes.flatten()[:2], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img / data.img_header['INT_TIME']
    flimg = rescale_intensity(1.0 * img)
    im = ax.imshow(flimg)
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
for ax, ind in zip(axes.flatten()[2:], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img / data.img_header['INT_TIME']
    flimg = rescale_intensity(1.0 * img)
    im = ax.imshow(equalize_hist(flimg))
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
fig.suptitle('Stretched between 0 and 1. And then equalized_histogram.')
plt.savefig('Stretched_both_ways.png', dpi=150)

In [101]:
pwd

'/home/klay6683/src/iuvs/notebooks'

In [97]:
plt.figure()
plt.hist(equalize_hist(flimg).flatten(), bins=100)

(array([  8.54670000e+05,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   3.16590000e+04,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   2.47800000e+04,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.76850000e+04,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.39610000e+04,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.18050000e+04,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   9.77800000e+03,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          7.86800000e+03,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   6.80400000e+03,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   6.12400000e+03,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

In [98]:
equalize_hist?